In [1]:
import sys
import terminator
import stanza
import nafigator

In [2]:
file = open('..//data//lassy//WORD-LEMMA-POS.freq', 'r', encoding="utf-8")
lines = file.readlines()

lassy = {}
for line in lines:
    line = line.strip()
    s = line.split(" ")
    data = s[1].split("\t")
    if len(data)==3:
        pos = data[2].split("(")[0].replace("N", "noun").lower()
        morphofeats = "("+data[2].split("(")[1]
        if (pos in ["adj", "noun"] or (pos =="spec") and morphofeats=="(afgebr)") and len(data[0])>1:
            if data[0] not in lassy.keys():
                lassy[data[0]] = [{'lemma': data[1], 
                                  'partOfSpeech': pos,
                                  'morphoFeats': morphofeats}]
            else:
                lassy[data[0]].append({'lemma': data[1], 
                                       'partOfSpeech': pos,
                                       'morphoFeats': morphofeats})

In [3]:
file = open('..//data//lassy//SOLVENCY2-LEMMA-POS.freq', 'r', encoding="utf-8")
lines = file.readlines()

for line in lines:
    data = line.split(", ")
    if len(data)==4:
        pos = data[3].split("(")[0].replace("N", "noun").lower()
        morphofeats = "("+data[3].split("(")[1][0:-1]
        if data[1] not in lassy.keys():
            lassy[data[1]] = [{'lemma': data[2], 
                              'partOfSpeech': pos,
                              'morphoFeats': morphofeats}]
        else:
            lassy[data[1]].append({'lemma': data[2], 
                                   'partOfSpeech': pos,
                                   'morphoFeats': morphofeats})

In [4]:
ref = terminator.TbxDocument()
params = {"sourceDesc": "TBX file, created via dnb/terminator"}
ref.generate(params)
for idx, concept_text in enumerate(lassy.keys()):
    concept = {
            "id": "c"+str(idx+1),
            "langSec": {
                "nl": [
                    {"type": "term", "text": concept_text}]}}
    for item in lassy[concept_text]:
        concept['langSec']['nl'].append(
                        {
                            "type": "termNote",
                            "attr": {"type": "lemma"},
                            "text": item['lemma']})
        concept['langSec']['nl'].append(
                        {
                            "type": "termNote",
                            "attr": {"type": "partOfSpeech"},
                            "text": item['partOfSpeech']})
        concept['langSec']['nl'].append(
                        {
                            "type": "termNote",
                            "attr": {"type": "morphoFeats"},
                            "text": item['morphoFeats']})

    ref.add_conceptEntry(concept=concept, params=params)

In [5]:
ref.write("..//data//lassy_with_insurance.tbx")